# <span style="color:#e67e22;">◼</span> **Project Overview**



This notebook is part of *Master Project 2025*, which focuses on assigning personalized perks to users based on their behavior on the travel platform. The analysis is entirely behavior-driven and deliberately excludes demographic or personal information. The ultimate goal is to segment users into meaningful groups and link each group with the most relevant perk.

Only perks that are supported by the available data are considered. These include:

- ✅ **Free checked bag**  
- ✅ **No cancellation fees**  
- ✅ **Exclusive discounts**  
- ✅ **One night free hotel with flight**  
- ❌ *Free hotel meal* (excluded due to missing data support)

This notebook covers the first steps of the analysis pipeline: loading and previewing raw data, selecting relevant variables, cleaning and transforming the datasets, and generating user-level features. These steps prepare the data for clustering, which will be used to assign the most suitable perk to each user segment.

# <span style="color:#e67e22;">◼</span> **EDA & Feature Engneering**

## <span style="color:#1f4e79;">◼</span> Load Data from AWS S3 Bucket

No accesdetails needed for this Project

In [26]:
import pandas as pd


def load_flights_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_flights_export_2025-03-31_134734.csv"
    return pd.read_csv(url)

def load_hotels_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_hotels_export_2025-03-31_171805.csv"
    return pd.read_csv(url)

def load_sessions_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_sessions_export_2025-03-31_221253.csv"
    return pd.read_csv(url)

def load_users_bronze():
    url = "https://lakehouse-masteryproject-2025.s3.eu-north-1.amazonaws.com/bronze/public_users_export_2025-04-01_101058.csv"
    return pd.read_csv(url)

In [27]:
df_flights_bronze = load_flights_bronze()
df_hotels_bronze = load_hotels_bronze()
df_sessions_bronze = load_sessions_bronze()
df_users_bronze = load_users_bronze()

print("✅ Flight, hotel, session and user data loaded successfully.")

✅ Flight, hotel, session and user data loaded successfully.


## <span style="color:#1f4e79;">◼</span> Reducing Columns

The next step is to reduce the columns to only those needed to validate the hypotheses behind the defined perks. All other columns are excluded unless they are required to calculate new features (e.g., `nights` for trip duration) or to support the interpretation of results.

**Personalization Scope (Project Context)**

- In this project, personalization is based exclusively on **individual user behavior** observed within the TravelTide platform. This includes patterns such as discount usage, booking behavior, and cancellation frequency.

- Demographic or group-level attributes (e.g. family status, number of travelers, or room count) are not considered. The goal is to assign each user the perk most relevant to **how they interact** with the platform — not who they are or who they travel with.



### <span style="color:#4b5320;">◼</span> Reducing Columns – `users` Table



For the `users` table, we keep only the columns required to support the behavioral logic behind the defined perks. Since personalization in this project is based purely on **individual interaction behavior**, we exclude all demographic and group-related attributes.


| Column         | Description                                                           | Keep? |
|----------------|------------------------------------------------------------------------|:-----:|
| `user_id`      | Required to join with sessions and flights                            | ✅    |
| `birthdate`    | Age not relevant to behavior-based logic                              | ❌    |
| `has_children` | Not used for any defined perk — group/family-related                  | ❌    |
| `gender`       | Not tied to any perk                                                  | ❌    |
| `married`      | Not tied to any perk                                                  | ❌    |
| `home_country` | Not required for perk logic                                           | ❌    |
| `home_city`    | Too granular for this scope                                           | ❌    |
| `home_airport` | Not relevant for current perks                                        | ❌    |
| `sign_up_date` | Not used in perk logic                                                | ❌    |


In [29]:
df_users_bronze = df_sessions_bronze[
    ['user_id']
]
df_users_bronze.head()

,user_id
0,841803
1,841804
2,841805
3,841806
4,841807


### <span style="color:#4b5320;">◼</span> Reducing Columns – `sessions` Table


The `sessions` table is the most important source for behavioral signals in this project. We retain only the columns directly required to support the five defined perks or essential to filtering and joining.

| Column                   | Description                                                                 | Keep? |
|--------------------------|-----------------------------------------------------------------------------|:-----:|
| `session_id`             | Needed to count sessions per user (7+ sessions filter)                      | ✅    |
| `user_id`                | Required to join with users                                                 | ✅    |
| `trip_id`                | Required to connect to flights and hotels                                   | ✅    |
| `session_start`          | Used to filter sessions after Jan 4, 2023                                   | ✅    |
| `flight_discount`        | Supports the "exclusive discounts" perk                                     | ✅    |
| `hotel_discount`         | Supports the "exclusive discounts" perk                                     | ✅    |
| `flight_discount_amount` | Captures price sensitivity for flight discounts                             | ✅    |
| `hotel_discount_amount`  | Captures price sensitivity for hotel discounts                              | ✅    |
| `flight_booked`          | Required to understand booking behavior                                     | ✅    |
| `hotel_booked`           | Required to validate hotel+flight pairing                                   | ✅    |
| `page_clicks`            | Indicates user engagement — helps understand interaction effort             | ✅    |
| `cancellation`           | Directly supports the "no cancellation fees" perk                           | ✅    |
| `session_end`            | Not required for any of the defined perks                                   | ❌    |


In [30]:
df_sessions_bronze = df_sessions_bronze[
    [
        "session_id", "user_id", "trip_id", "session_start",
        "flight_discount", "hotel_discount",
        "flight_discount_amount", "hotel_discount_amount",
        "flight_booked", "hotel_booked",
        "page_clicks", "cancellation"
    ]
]
df_sessions_bronze.head()

,session_id,user_id,trip_id,session_start,flight_discount,hotel_discount,flight_discount_amount,hotel_discount_amount,flight_booked,hotel_booked,page_clicks,cancellation
0,841803-95c93659053948049c73106ffdcf4df6,841803,841803-31d33d39b08f477bb8d9dd3b0dcbf1d3,2023-05-17 19:40:00,False,False,NaN,NaN,True,True,30,False
1,841804-bf1bfe452c4d4372b04d9612134006e7,841804,NaN,2023-05-17 22:40:00,False,False,NaN,NaN,False,False,9,False
2,841805-70d695ee6540481bae7750e368c10443,841805,841805-506d17340ffa41659736fd258138a9b5,2023-05-17 21:10:00,False,False,NaN,NaN,True,True,24,False
3,841806-a03ac981d1c847979765669ca871a0cb,841806,NaN,2023-05-17 21:49:00,True,True,0.2,0.2,False,False,42,False
4,841807-4febb023cd414a6196fc026af1928bfb,841807,NaN,2023-05-17 20:28:00,False,False,NaN,NaN,False,False,4,False


### <span style="color:#4b5320;">◼</span> Reducing Columns – `flights` Table



We retain only the columns that are directly required to support perk validation. Columns related to destinations, time, or route details are excluded, as none of the five defined perks depend on them.

| Column                   | Description                                                                                  |
|--------------------------|----------------------------------------------------------------------------------------------|
| `trip_id`                | ✅ Required to join with sessions                                                             |
| `checked_bags`           | ✅ Directly supports the "free checked bag" perk                                              |
| `base_fare_usd`          | ✅ Supports the "exclusive discounts" perk by capturing price sensitivity                     |
| `trip_airline`           | ❌ Not needed for current perks                                                               |
| `origin_airport`         | ❌ Not required for any defined perk                                                          |
| `destination`            | ❌ Not required for any defined perk                                                          |
| `destination_airport`    | ❌ Redundant and not perk-related                                                             |
| `seats`                  | ❌ Not needed for defined perks                                                               |
| `return_flight_booked`   | ❌ Not used for any current perk                                                              |
| `departure_time`         | ❌ Not used for any current perk                                                              |
| `return_time`            | ❌ Not used for any current perk                                                              |
| `destination_airport_lat`| ❌ Mapping only — not used in perks                                                           |
| `destination_airport_lon`| ❌ Mapping only — not used in perks                                                           |


In [31]:
df_flights_bronze = df_flights_bronze[
    ["trip_id", "checked_bags", "base_fare_usd"]
]

df_flights_bronze.head()


,trip_id,checked_bags,base_fare_usd
0,693092-7b8767746abd45219ed468d58da9f702,0,479.31
1,693124-1032149a9e01427da914b17ce6ef1926,0,575.78
2,693149-0fdb31c7dc114a4d8a318f0feada365c,0,336.72
3,693169-d120c3721d0b4bcca278639c981e4322,0,295.37
4,693181-f4a2363db0ee452f8ac8c0fe3d4ef0ad,1,694.26


### <span style="color:#4b5320;">◼</span> Reducing Columns – `hotels` Table

In the `hotels` table, we only keep columns that are directly needed to validate the defined perks. All other fields, such as room count, hotel brand, or check-in/out times, are excluded since they are not required for any of the five perks.


| Column               | Description                                                                                   |
|----------------------|-----------------------------------------------------------------------------------------------|
| `trip_id`            | ✅ Required to join hotels with sessions and flights                                          |
| `nights`             | ✅ Required to validate the "1 night free hotel with flight" perk                             |
| `hotel_per_room_usd` | ✅ Supports the "exclusive discounts" perk by capturing price sensitivity                      |
| `hotel_name`         | ❌ Not needed for any defined perk                                                             |
| `rooms`              | ❌ Not used for any current perk                                                               |
| `check_in_time`      | ❌ Not required for any of the defined perks                                                   |
| `check_out_time`     | ❌ Not required — duration is already captured by `nights`                                     |


Instead of relying on the nights column from the dataset, we calculate it ourselves based on actual hote logic: check-in time, check-out time, and a grace period. A late checkout may result in an additional night being counted. This behavior is more realistic.

In [32]:
df_hotels_bronze = df_hotels_bronze.drop(columns=["nights"])

In [33]:
from datetime import datetime, timedelta
import pandas as pd

# Hotel rules
CHECKIN_TIME = datetime.strptime("15:00", "%H:%M").time()
CHECKOUT_TIME = datetime.strptime("11:00", "%H:%M").time()
GRACE_PERIOD = timedelta(hours=1)  # 1-hour grace period

# Convert check-in/out columns to datetime first
df_hotels_bronze["check_in_time"] = pd.to_datetime(df_hotels_bronze["check_in_time"], errors="coerce")
df_hotels_bronze["check_out_time"] = pd.to_datetime(df_hotels_bronze["check_out_time"], errors="coerce")

# Function to calculate realistic nights
def calculate_nights_realistic(row):
    check_in = row["check_in_time"]
    check_out = row["check_out_time"]

    if pd.isna(check_in) or pd.isna(check_out):
        return None

    check_in_date = check_in.date()
    check_out_date = check_out.date()
    nights = (check_out_date - check_in_date).days

    # Add 1 night if checkout was late (after grace period)
    checkout_deadline = datetime.combine(check_out_date, CHECKOUT_TIME) + GRACE_PERIOD
    if check_out > checkout_deadline:
        nights += 1

    return max(nights, 0)  # Ensure non-negative result

# Recalculate nights column
df_hotels_bronze["nights"] = df_hotels_bronze.apply(calculate_nights_realistic, axis=1)


In [34]:
df_hotels_bronze = df_hotels_bronze[["trip_id", "nights", "hotel_per_room_usd"]]

df_hotels_bronze.head()

,trip_id,nights,hotel_per_room_usd
0,234666-99574aa53f30402ba04bfe2f3337c566,2.0,226
1,234707-4c0cfdb5ff4a4cd8bda5feed22f790d7,2.0,114
2,234745-a8f944dc121c4003b16c108c9c590965,4.0,322
3,234905-c1e14087ef1f4a7a8413a12ef4208135,2.0,228
4,235101-cd0e752033db4f488fa75b5eb8feb2c8,2.0,163


## <span style="color:#1f4e79;">◼</span> Parse and Standardize Data Types

Before we can assign perks or create user-level aggregations, we need to ensure that all four tables — `users`, `sessions`, `flights`, and `hotels` — are clean and consistent. This includes standardizing data types, handling missing values, and ensuring that binary and datetime fields are correctly formatted.

Each table requires slightly different steps, depending on the structure of the data it contains.

### <span style="color:#4b5320;">◼</span> Data Preparation – `users` Table

The `users` table contains only two columns needed for the defined perks: `user_id` and `birthdate`. Our main goal here is to ensure `birthdate` is in datetime format so we can calculate user age if needed.


#### <span style="color:#800020;">◼</span> Convert Columns to Correct Data Types


In [10]:
df_users_bronze["birthdate"] = pd.to_datetime(df_users_bronze["birthdate"], errors="coerce")

#### <span style="color:#800020;">◼</span> Handle Missing Values

In [11]:
# Check for nulls
df_users_bronze.isna().sum()

# Drop rows with missing user_id or birthdate
df_users_bronze = df_users_bronze.dropna(subset=["user_id", "birthdate"])


In [12]:
df_users_bronze.head()

,user_id,birthdate
0,0,1990-01-22
1,1,2000-11-08
2,2,1992-09-21
3,3,1996-11-27
4,4,1978-01-05


### <span style="color:#4b5320;">◼</span> Data Preparation – `sessions` Table

We start by preparing the `sessions` table. The main focus is on:

- Ensuring correct data types (especially for datetime and boolean fields)
- Handling missing values
- Verifying binary logic for downstream scoring and aggregation

#### <span style="color:#800020;">◼</span> Convert Columns to Correct Data Types

In [13]:
df_sessions_bronze["session_start"] = pd.to_datetime(df_sessions_bronze["session_start"])
df_sessions_bronze["flight_discount"] = df_sessions_bronze["flight_discount"].astype(bool)
df_sessions_bronze["hotel_discount"] = df_sessions_bronze["hotel_discount"].astype(bool)
df_sessions_bronze["flight_booked"] = df_sessions_bronze["flight_booked"].astype(bool)
df_sessions_bronze["hotel_booked"] = df_sessions_bronze["hotel_booked"].astype(bool)
df_sessions_bronze["cancellation"] = df_sessions_bronze["cancellation"].astype(bool)

#### <span style="color:#800020;">◼</span> Handle Missing Values

In [14]:
# Check for nulls
df_sessions_bronze.isna().sum()

# Drop sessions without user_id or trip_id, which are critical
df_sessions_bronze = df_sessions_bronze.dropna(subset=["user_id", "trip_id"])

# Fill missing discount amounts with 0
df_sessions_bronze["flight_discount_amount"] = df_sessions_bronze["flight_discount_amount"].fillna(0)
df_sessions_bronze["hotel_discount_amount"] = df_sessions_bronze["hotel_discount_amount"].fillna(0)


#### <span style="color:#800020;">◼</span> Validate Binary Columns

In [15]:
# Confirm binary fields contain only True/False
binary_fields = ["flight_discount", "hotel_discount", "flight_booked", "hotel_booked", "cancellation"]
for col in binary_fields:
    print(col, df_sessions_bronze[col].unique())


flight_discount [False  True]
hotel_discount [False  True]
flight_booked [ True False]
hotel_booked [ True False]
cancellation [False  True]


### <span style="color:#4b5320;">◼</span> Data Preparation – `flights` Table

In the `flights` table, we prepare the data by ensuring correct data types and handling missing values in price and baggage-related columns. Since only three columns are needed to validate the defined perks, we focus cleanup on those specifically.


#### <span style="color:#800020;">◼</span> Convert Columns to Correct Data Types

In [16]:
df_flights_bronze["checked_bags"] = pd.to_numeric(df_flights_bronze["checked_bags"], errors="coerce")
df_flights_bronze["base_fare_usd"] = pd.to_numeric(df_flights_bronze["base_fare_usd"], errors="coerce")

#### <span style="color:#800020;">◼</span> Handle Missing Values

In [17]:
# Check for nulls
df_flights_bronze.isna().sum()

# Drop rows missing critical fields
df_flights_bronze = df_flights_bronze.dropna(subset=["trip_id", "checked_bags", "base_fare_usd"])

### <span style="color:#4b5320;">◼</span> Data Preparation – `hotels` Table

In the `hotels` table, we focus on ensuring that time and price fields are correctly formatted and that missing values are handled. This is especially important for calculating `nights` from `check_in_time` and `check_out_time`.


#### <span style="color:#800020;">◼</span> Convert Columns to Correct Data Types

In [18]:
df_hotels_bronze["nights"] = pd.to_numeric(df_hotels_bronze["nights"], errors="coerce")
df_hotels_bronze["hotel_per_room_usd"] = pd.to_numeric(df_hotels_bronze["hotel_per_room_usd"], errors="coerce")

#### <span style="color:#800020;">◼</span> Handle Missing Values

In [19]:
# Check for nulls
df_hotels_bronze.isna().sum()

# Drop rows with missing trip_id or calculated nights
df_hotels_bronze = df_hotels_bronze.dropna(subset=["trip_id", "nights"])


#### <span style="color:#800020;">◼</span> Export Silver Tables for Manual AWS S3 Upload

In [20]:
import pandas as pd
import os

# Silver DataFrames (copies of bronze, after cleaning)
df_flights_silver = df_flights_bronze
df_hotels_silver = df_hotels_bronze
df_sessions_silver = df_sessions_bronze
df_users_silver = df_users_bronze

# Folder to save locally before manual S3 upload
silver_local_path = "silver_exports/"
os.makedirs(silver_local_path, exist_ok=True)

# Tables to save
tables = {
    "flights_silver": df_flights_silver,
    "users_silver": df_users_silver,
    "hotels_silver": df_hotels_silver,
    "sessions_silver": df_sessions_silver
}

# Save each table as CSV locally
for name, df in tables.items():
    try:
        path = os.path.join(silver_local_path, f"{name}.csv")
        df.to_csv(path, index=False)
        print(f"✅ Exported: {path}")
    except Exception as e:
        print(f"❌ Error in {name}: {e}")


✅ Exported: silver_exports/flights_silver.csv
✅ Exported: silver_exports/users_silver.csv
✅ Exported: silver_exports/hotels_silver.csv
✅ Exported: silver_exports/sessions_silver.csv


## <span style="color:#1f4e79;">◼</span> Users with ≥ 7 Sessions Since January 4, 2023

This threshold is defined in the official project guidelines. Users with fewer than 7 sessions since January 4, 2023 are excluded to ensure that only active users with sufficient interaction history are included in the analysis.

In [21]:
import pandas as pd

# 1. Filter sessions since January 4, 2023
df_sessions_silver["session_start"] = pd.to_datetime(df_sessions_silver["session_start"])
recent_sessions = df_sessions_silver[df_sessions_silver["session_start"] >= "2023-01-04"]

# 2. Count sessions per user and select those with ≥ 7
session_counts = recent_sessions["user_id"].value_counts()
eligible_user_ids = session_counts[session_counts >= 7].index

# 3. Filter all silver tables to include only eligible users
df_sessions_silver = df_sessions_silver[df_sessions_silver["user_id"].isin(eligible_user_ids)]
df_users_silver = df_users_silver[df_users_silver["user_id"].isin(eligible_user_ids)]

# 4. Keep only trips linked to eligible users for hotel/flight data
valid_trip_ids = df_sessions_silver["trip_id"].dropna().unique()
df_flights_silver = df_flights_silver[df_flights_silver["trip_id"].isin(valid_trip_ids)]
df_hotels_silver = df_hotels_silver[df_hotels_silver["trip_id"].isin(valid_trip_ids)]

print(f"✅ Filtered to cohort: {len(eligible_user_ids)} users with ≥ 7 sessions since 2023-01-04.")

✅ Filtered to cohort: 79 users with ≥ 7 sessions since 2023-01-04.


# <span style="color:#e67e22;">◼</span> **Customer Segmentation**

Now that the data is cleaned and all perk-relevant features have been engineered, we begin the segmentation phase. The goal is to group customers based on their behavior and assign the most suitable perk to each group.


## <span style="color:#1f4e79;">◼</span> Clustering Pipeline Overview

To assign the most suitable perk to each user based on behavior, the following five-step clustering process is applied:




1. Aggregate user-level metrics  
   ○ Create behavioral features (e.g., sessions, cancellations, discounts)  

2. Scale features for clustering  
   ○ Normalize numeric values (e.g., MinMaxScaler)  

3. Analyze clustering logic & choose method  
   ○ Check feature correlation and determine optimal number of clusters  

4. Apply clustering algorithm  
   ○ Use K-Means to segment users based on behavior  

5. Interpret clusters & assign perks  
   ○ Match perk logic to the dominant patterns in each group  


### <span style="color:#4b5320;">◼</span> Step 1 – User-Level Aggregation

To segment customers meaningfully, we first create a single user-level DataFrame that aggregates all relevant behavioral metrics from the cleaned `silver` tables.

These features will serve as input for clustering.

**Features to Aggregate Per User**
| Feature                     | Source Table        | Description                                                                 |
|-----------------------------|---------------------|-----------------------------------------------------------------------------|
| `total_sessions`           | sessions_silver     | Total number of sessions per user                                           |
| `cancellation_rate`        | sessions_silver     | Share of sessions marked as cancellations                                   |
| `discount_usage_rate`      | sessions_silver     | Share of sessions where any discount was applied                            |
| `total_nights`             | hotels_silver       | Total number of nights across all hotel bookings                            |
| `total_checked_bags`       | flights_silver      | Total number of checked bags across all flights                             |
| `total_base_fare`          | flights_silver      | Sum of base flight fares — indicates price sensitivity or booking class     |


#### <span style="color:#800020;">◼</span> Aggregation Code – Create `df_user_features`

In [22]:
# ✅ Create a copy to avoid SettingWithCopyWarning
df_sessions_silver = df_sessions_silver.copy()

# 🧩 Add helper column: True if any discount was used in a session
df_sessions_silver["used_any_discount"] = df_sessions_silver["flight_discount"] | df_sessions_silver["hotel_discount"]

# 📊 Aggregate session-level features per user
sessions_agg = df_sessions_silver.groupby("user_id").agg(
    total_sessions=("session_id", "count"),
    cancellation_rate=("cancellation", "mean"),
    discount_usage_rate=("used_any_discount", "mean")
).reset_index()


In [23]:
# --- Prepare sessions ---
# Add helper column: True if user used any discount (flight or hotel)
df_sessions_silver.loc[:, "used_any_discount"] = (
    df_sessions_silver["flight_discount"] | df_sessions_silver["hotel_discount"]
)

# Aggregate session-based metrics
sessions_agg = df_sessions_silver.groupby("user_id").agg(
    total_sessions=("session_id", "count"),
    cancellation_rate=("cancellation", "mean"),
    discount_usage_rate=("used_any_discount", "mean")
)

# --- Prepare hotels ---
hotels_agg = df_hotels_silver.groupby("trip_id")["nights"].sum().reset_index()
hotels_sessions = df_sessions_silver[["user_id", "trip_id"]].drop_duplicates()
hotels_joined = hotels_sessions.merge(hotels_agg, on="trip_id", how="left")
hotel_user_agg = hotels_joined.groupby("user_id")["nights"].sum().rename("total_nights")

# --- Prepare flights ---
flights_agg = df_flights_silver.groupby("trip_id").agg(
    total_checked_bags=("checked_bags", "sum"),
    total_base_fare=("base_fare_usd", "sum")
).reset_index()
flights_sessions = df_sessions_silver[["user_id", "trip_id"]].drop_duplicates()
flights_joined = flights_sessions.merge(flights_agg, on="trip_id", how="left")
flight_user_agg = flights_joined.groupby("user_id").agg(
    total_checked_bags=("total_checked_bags", "sum"),
    total_base_fare=("total_base_fare", "sum")
)

# --- Merge all user-level features ---
df_user_features = (
    sessions_agg
    .join(hotel_user_agg, how="left")
    .join(flight_user_agg, how="left")
    .fillna(0)
    .reset_index()
)

print("✅ User-level features aggregated.")
df_user_features.head()

✅ User-level features aggregated.


,user_id,total_sessions,cancellation_rate,discount_usage_rate,total_nights,total_checked_bags,total_base_fare
0,144844,9,0.333333,0.444444,35.0,2.0,7827.97
1,204943,10,0.100000,0.200000,40.0,8.0,3940.79
2,260230,10,0.100000,0.300000,18.0,6.0,2911.17
3,290123,8,0.000000,0.000000,22.0,7.0,4939.42
4,405311,8,0.125000,0.625000,19.0,2.0,2175.21


#### <span style="color:#800020;">◼</span> Histogram: Total Sessions per User

In [24]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
df_users['total_sessions'].hist(bins=30, edgecolor='black')
plt.title('Distribution of Total Sessions per User')
plt.xlabel('Total Sessions')
plt.ylabel('User Count')
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'df_users' is not defined

<Figure size 800x500 with 0 Axes>

#### <span style="color:#800020;">◼</span> Boxplot: Flight Base Fare (USD)

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 4))
sns.boxplot(x=df_flights['base_fare_usd'], color='skyblue')
plt.title('Boxplot of Flight Base Fare (USD)')
plt.xlabel('Base Fare (USD)')
plt.tight_layout()
plt.show()


NameError: name 'df_flights' is not defined

<Figure size 800x400 with 0 Axes>

### <span style="color:#4b5320;">◼</span> Step 2 – Feature Scaling for Clustering

To ensure all features contribute equally to the clustering process, we standardize selected user-level metrics using `MinMaxScaler`. This transforms all values into a uniform range between 0 and 1.

**Features scaled:**
- `total_sessions`
- `cancellation_rate`
- `discount_usage_rate`
- `total_nights`
- `total_checked_bags`
- `total_base_fare`

The scaled values are stored in a new DataFrame (`df_scaled`) and are now ready for use in the clustering algorithm.


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 🧮 Features to scale
features_to_scale = [
    "total_sessions",
    "cancellation_rate",
    "discount_usage_rate",
    "total_nights",
    "total_checked_bags",
    "total_base_fare"
]

# 🔁 Standardize values to [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_user_features[features_to_scale])

# 💾 Store scaled features in new DataFrame
df_scaled = pd.DataFrame(X_scaled, columns=features_to_scale)
df_scaled["user_id"] = df_user_features["user_id"].values

print("✅ Scaled features ready for clustering.")
df_scaled.head()


#### <span style="color:#800020;">◼</span> Export Gold Tables for Manual AWS S3 Upload

In [ ]:
import pandas as pd
import os

# Gold DataFrames (filtered for cohort and ready for clustering)
df_flights_gold = df_flights_silver
df_hotels_gold = df_hotels_silver
df_sessions_gold = df_sessions_silver
df_users_gold = df_users_silver

# Folder to save locally before manual S3 upload
gold_local_path = "gold_exports/"
os.makedirs(gold_local_path, exist_ok=True)

# Tables to save
tables = {
    "flights_gold": df_flights_gold,
    "users_gold": df_users_gold,
    "hotels_gold": df_hotels_gold,
    "sessions_gold": df_sessions_gold,
    "user_features_gold": df_user_features  # Newly added gold-layer export
}

# Save each table as CSV locally
for name, df in tables.items():
    try:
        path = os.path.join(gold_local_path, f"{name}.csv")
        df.to_csv(path, index=False)
        print(f"✅ Exported: {path}")
    except Exception as e:
        print(f"❌ Error in {name}: {e}")


### <span style="color:#4b5320;">◼</span> Step 3 – Clustering the Users

**What we’ll do in this step:**

- Decide how many clusters to try (e.g. k=3, k=4, k=5)

- Fit the KMeans model on df_scaled (without user_id)

- Store the assigned cluster label back in df_user_features

#### <span style="color:#800020;">◼</span> Feature Correlation Matrix – Identify Redundancies Before Clustering

In [37]:
import seaborn as sns
import matplotlib.pyplot as plt

# 🔍 Only numeric features for correlation
corr_data = df_user_features.drop(columns=["user_id", "cluster"])

# 📊 Plot the correlation matrix
plt.figure(figsize=(10, 6))
sns.heatmap(corr_data.corr(), annot=True, cmap="YlGnBu", linewidths=0.5)
plt.title("Correlation Matrix – User-Level Features")
plt.tight_layout()
plt.show()


KeyError: "['cluster'] not found in axis"

No strong multicollinearity → we don’t need to drop or merge features due to redundancy.

The features are informationally diverse, which is great for clustering – K-Means can now form groups based on genuinely different behaviors.

#### <span style="color:#800020;">◼</span> Determine Optimal `k` – Elbow Curve for K-Means

We use the elbow method to plot inertia (clustering error) for a range of cluster counts.  
The "elbow point" indicates a good balance between cluster compactness and generalization.

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Drop user_id column before clustering
X = df_scaled.drop(columns=["user_id"])

# Try different k values
inertia = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(8, 4))
plt.plot(k_range, inertia, marker="o")
plt.title("Elbow Curve – K-Means Inertia vs. Cluster Count")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.grid(True)
plt.tight_layout()
plt.show()

### <span style="color:#4b5320;">◼</span> Step 4 – Apply KMeans Clustering

We apply KMeans clustering with `k = 4` and assign each user a cluster label for further analysis.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 🎯 Drop user_id for clustering
X = df_scaled.drop(columns=["user_id"])

# 🔢 Set number of clusters from elbow method
k = 4  # ⬅️ ggf. anpassen

# 🚀 Apply K-Means clustering
kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
df_scaled["cluster"] = kmeans.fit_predict(X)

# 🔄 Add cluster labels to original user features
df_user_features["cluster"] = df_scaled["cluster"]

# 🎨 PCA for 2D visualization
pca = PCA(n_components=2)
components = pca.fit_transform(X)

# 📊 Plot PCA projection with clusters
plt.figure(figsize=(8, 5))
scatter = plt.scatter(
    components[:, 0], components[:, 1],
    c=df_scaled["cluster"],
    cmap="Set2",
    s=40,
    edgecolor='k'
)
plt.title(f"User Segmentation – PCA Projection (k={k})")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.grid(True)
plt.legend(*scatter.legend_elements(), title="Cluster")
plt.tight_layout()
plt.show()


**What I think is happening:**<br>
<br>
- The users on the left (green) are one type – maybe low spenders or low activity.

- The users on the right (blue) seem very different – maybe they book a lot or spend more.

- The yellow ones are more spread out, so maybe they’re a mixed group.

- The gray ones are very tight and close – maybe their behavior is really similar.

- I can't say exactly what each group means just by looking at the chart, so I plan to check the average behavior per cluster next.

### <span style="color:#4b5320;">◼</span> Step 5 – Interpret Clusters & Assign Perks

In [ ]:
# Calculate cluster-wise means from the scaled data
scaled_means = df_scaled.drop(columns="user_id").groupby("cluster").mean()

# Transpose the DataFrame for better plotting (features on x-axis)
scaled_means.T.plot(kind="bar", figsize=(10, 6), colormap="Set2")

plt.title("Average Scaled Feature Values per Cluster")
plt.ylabel("Scaled Value (0–1)")
plt.xlabel("Feature")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


**Cluster Interpretation (based on the bar chart)**

| Cluster    | What I see |
|------------|------------|
| **Cluster 0** | These users have **high checked bags** and **high base fares**, but **few sessions**. Maybe they travel rarely, but when they do, they **spend more** and **bring more luggage**. |
| **Cluster 1** | They use **a lot of discounts**, have **medium cancellation rates**, and **spend less overall**. Sounds like **budget-conscious travelers** who book carefully. |
| **Cluster 2** | This group is **high on almost everything** – sessions, nights, bags, fare. They look like **power users**, but also **cancel the most**. Maybe they book a lot but often change plans. |
| **Cluster 3** | This group is **low to medium on all features**. Not much stands out. They seem like **average or low-engagement users**. |


We now examine each cluster’s average behavior to determine which **perk** fits best. This is based on aggregated metrics like discount usage, cancellations, and travel activity. We calculate mean feature values for each cluster to detect dominant patterns.

In [ ]:
# Calculate average behavior per cluster
cluster_summary = df_user_features.groupby("cluster").mean(numeric_only=True)
cluster_summary.style.background_gradient(cmap="YlGnBu").format("{:.2f}")


**Cluster Interpretation & Perk Assignment**
| Cluster | Behavioral Summary                                                                 | Assigned Perk                  |
|---------|-------------------------------------------------------------------------------------|--------------------------------|
| 0       | High discount usage, low cancellations, active in both flights and hotels          | 🎁 Exclusive Discounts         |
| 1       | Low spenders, few sessions, little engagement                                      | 🧳 Free Checked Bag            |
| 2       | High cancellation rate, medium session volume                                      | ❌ No Cancellation Fees        |
| 3       | Frequent hotel+flight usage, long nights, high fares                               | 🏨 1 Night Free with Flight    |


# <span style="color:#e67e22;">◼</span> **Machine Learning - Model**

We train a classifier to predict user clusters from behavioral features.  
This allows perk recommendations to be extended to new users in a scalable, supervised way.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Step 1 – Features (X) and target (y)
X = df_user_features[[
    "total_sessions",
    "cancellation_rate",
    "discount_usage_rate",
    "total_nights",
    "total_checked_bags",
    "total_base_fare"
]]

y = df_user_features["cluster"]

# Step 2 – Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 3 – Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4 – Evaluate
y_pred = clf.predict(X_test)
print("✅ Classification Report:")
print(classification_report(y_test, y_pred))

print("🧩 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


🎯 Random Forest Classification to Validate Clusters

I trained a **Random Forest model** to predict which cluster each user belongs to, using the following features:

- `total_sessions`
- `cancellation_rate`
- `discount_usage_rate`
- `total_nights`
- `total_checked_bags`
- `total_base_fare`

I used a **train-test split (80/20)** and evaluated the model on the test set.

---

✅ Classification Report Summary

- **Accuracy**: 94%
- **Precision / Recall / F1** scores are all strong, especially for clusters 1, 2, and 3.
- Cluster 0 had one misclassified user (predicted as Cluster 1).
- All other clusters were predicted correctly.

---

🧩 Confusion Matrix

| Actual \ Predicted | Cluster 0 | Cluster 1 | Cluster 2 | Cluster 3 |
|--------------------|-----------|-----------|-----------|-----------|
| **Cluster 0**      | 2         | 1         | 0         | 0         |
| **Cluster 1**      | 0         | 5         | 0         | 0         |
| **Cluster 2**      | 0         | 0         | 1         | 0         |
| **Cluster 3**      | 0         | 0         | 0         | 7         |

---

**Cluster Validation – Linking Classification to Perk Assignment**

| Cluster | Classification Accuracy | Behavioral Summary (from KMeans)                          | Assigned Perk                |
|---------|--------------------------|------------------------------------------------------------|------------------------------|
| 0       | 2/3 correct (1 misclass.) | Few sessions, high fares and luggage                       | 🎁 Exclusive Discounts       |
| 1       | 5/5 correct               | Budget-conscious, medium cancellations, discounts          | 🧳 Free Checked Bag          |
| 2       | 1/1 correct               | High on all metrics, cancels a lot (power users)           | ❌ No Cancellation Fees      |
| 3       | 7/7 correct               | Average to low usage                                       | 🏨 1 Night Free with Flight  |


**Cluster Validation & Perk Confirmation**

To validate whether the clusters derived from the K-Means algorithm truly reflect distinct user behavior, I trained a Random Forest classifier. The goal was to predict each user's cluster assignment based solely on the six aggregated behavioral features that were also used for clustering.

Using an 80/20 train-test split, the model achieved an accuracy of 94%. All clusters were predicted correctly except for one user from Cluster 0, who was misclassified as Cluster 1.

This strong performance indicates that the clustering is not random but based on consistent patterns in user behavior. Since each cluster was mapped to a specific perk, the classification results also support the logic behind the perk assignments.